In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
df = pd.read_csv('MTA_Subway_Hourly_Ridership__Beginning_February_2022_20240522.csv')

C:\Users\Daniel\AppData\Local\Temp\ipykernel_12400\1196061115.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('MTA_Subway_Hourly_Ridership__Beginning_February_2022_20240522.csv')


In [3]:
df.head()

,transit_timestamp,transit_mode,station_complex_id,station_complex,borough,payment_method,fare_class_category,ridership,transfers,latitude,longitude,Georeference
0,05/24/2023 06:00:00 AM,subway,61,50 St (D),Brooklyn,omny,OMNY - Full Fare,35,2,40.636261,-73.994789,POINT (-73.99478912353516 40.636260986328125)
1,03/20/2023 01:00:00 PM,subway,61,50 St (D),Brooklyn,omny,OMNY - Full Fare,29,2,40.636261,-73.994789,POINT (-73.99478912353516 40.636260986328125)
2,01/21/2023 01:00:00 PM,subway,610,"Grand Central-42 St (S,4,5,6,7)",Manhattan,metrocard,Metrocard - Full Fare,636,9,40.751778,-73.976845,POINT (-73.97684478759766 40.75177764892578)
3,05/24/2023 06:00:00 PM,subway,610,"Grand Central-42 St (S,4,5,6,7)",Manhattan,metrocard,Metrocard - Fair Fare,149,0,40.751778,-73.976845,POINT (-73.97684478759766 40.75177764892578)
4,06/29/2023 02:00:00 AM,subway,610,"Grand Central-42 St (S,4,5,6,7)",Manhattan,metrocard,Metrocard - Fair Fare,3,0,40.751778,-73.976845,POINT (-73.97684478759766 40.75177764892578)


Get a csv file of longitudes and latitudes for each station complex

In [9]:
mode_values = df.groupby(['station_complex_id']).agg(lambda x: pd.Series.mode(x)[0]).reset_index()

In [10]:
mode_values

,station_complex_id,transit_timestamp,transit_mode,station_complex,borough,payment_method,fare_class_category,ridership,transfers,latitude,longitude,Georeference
0,1,09/28/2022 03:00:00 PM,subway,"Astoria-Ditmars Blvd (N,W)",Queens,metrocard,Metrocard - Unlimited 7-Day,1,0,40.775036,-73.912033,POINT (-73.91203308105469 40.7750358581543)
1,2,07/25/2022 10:00:00 AM,subway,"Astoria Blvd (N,W)",Queens,metrocard,Metrocard - Full Fare,1,0,40.770260,-73.917847,POINT (-73.9178466796875 40.770259857177734)
2,3,02/14/2022 12:00:00 PM,subway,"30 Av (N,W)",Queens,metrocard,Metrocard - Full Fare,1,0,40.766781,-73.921478,POINT (-73.92147827148438 40.766780853271484)
3,4,02/23/2022 06:00:00 PM,subway,"Broadway (N,W)",Queens,metrocard,Metrocard - Full Fare,1,0,40.761822,-73.925507,POINT (-73.92550659179688 40.76182174682617)
4,5,02/21/2022 03:00:00 PM,subway,"36 Av (N,W)",Queens,metrocard,Metrocard - Full Fare,1,0,40.756805,-73.929573,POINT (-73.92957305908203 40.756805419921875)
...,...,...,...,...,...,...,...,...,...,...,...,...
849,97,01/30/2024 03:00:00 PM,subway,"Myrtle Av (M,J,Z)",Brooklyn,metrocard,Metrocard - Unlimited 7-Day,1,0,40.697208,-73.935654,POINT (-73.93565368652344 40.697208404541016)
850,98,01/03/2024 03:00:00 PM,subway,"Flushing Av (M,J)",Brooklyn,metrocard,OMNY - Full Fare,1,0,40.700260,-73.941124,POINT (-73.94112396240234 40.700260162353516)
851,99,01/30/2024 02:00:00 PM,subway,"Lorimer St (M,J)",Brooklyn,metrocard,OMNY - Full Fare,1,0,40.703869,-73.947411,POINT (-73.9474105834961 40.7038688659668)
852,TRAM1,02/28/2024 09:00:00 AM,tram,RI Tramway (Manhattan),Manhattan,metrocard,Metrocard - Full Fare,1,0,40.761338,-73.964155,POINT (-73.964155 40.761338)


In [11]:
mode_values = mode_values[['station_complex_id', 'station_complex', 'borough', 'payment_method', 'latitude', 'longitude']]

In [12]:
mode_values.to_csv("station_complex_information.csv", index=False)

Drop unneeded columns

In [14]:
df.drop(['transit_mode', 'station_complex', 'payment_method', 'fare_class_category', 'transfers', 'Georeference'],axis=1, inplace=True)

Group by hour

In [20]:
grouped_df = df.groupby(['transit_timestamp', 'station_complex_id'])['ridership'].sum().reset_index()

In [23]:
grouped_df.shape

(8371645, 3)

In [24]:
df.shape

(57291332, 6)

In [25]:
df['transit_timestamp'].nunique()

19891

In [30]:
grouped_df['station_complex_id'] = grouped_df['station_complex_id'].astype(str)

In [31]:
grouped_df.to_parquet("mta_subway_cleaned.parquet", engine="pyarrow", index=False)